<a href="https://colab.research.google.com/github/MrFlygerian/NLP-Document-Summary/blob/master/Document_summariser_(experiment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!ls "/content/drive/My Drive/Colab Notebooks"
%cd "/content/drive/My Drive/Colab Notebooks"

 control-weights-30-1.5048.hdf5		   __pycache__
'Document Summariser (control).ipynb'	   README.md
'Document summariser (experiment).ipynb'   starting-weight.hdf5
 get_docx_text.py			   wonderland.txt
/content/drive/My Drive/Colab Notebooks


In [29]:
#specific to extracting information from word documents
from get_docx_text import get_docx_text
import os
import sys
#converting letters to numbers
import numpy as np

#deep learning modules
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [30]:
#load files and extract text

#move to the file container folder
%cd '/content/drive/My Drive/Files from retreat/'

files = [f for f in os.listdir('.')]
texts = []
for file in files:
  try:
    texts.append(get_docx_text(file).lower())
  except:
    continue
train_text = ' '.join(texts)

#move back to working directory
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Files from retreat
/content/drive/My Drive/Colab Notebooks


In [31]:
#create numberical map using document characters
chars = sorted(list(set(train_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [32]:
# summarize the loaded data
n_chars = len(train_text)
n_vocab = len(chars)

In [33]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = train_text[i:i + seq_length]
	seq_out = train_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)

In [34]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [35]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.load_weights('control-weights-30-1.5048.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [36]:
#define checkpoint
filepath="experiment-weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
#fit model
model.fit(X, y, epochs=30, batch_size=128, callbacks=callbacks_list)

Epoch 1/30
105133/105133 [==============================] - 489s 5ms/step - loss: 1.4919

Epoch 00001: loss improved from inf to 1.49187, saving model to experiment-weights-01-1.4919.hdf5
Epoch 2/30
105133/105133 [==============================] - 483s 5ms/step - loss: 1.4705

Epoch 00002: loss improved from 1.49187 to 1.47054, saving model to experiment-weights-02-1.4705.hdf5
Epoch 3/30
 61056/105133 [================>.............] - ETA: 3:19 - loss: 1.4471

In [ ]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


In [ ]:
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")